In [1]:
import sys
#change it to your own path to where 'gym' is installed
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages')
import random
import gym
import numpy as np

from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

#from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display
from multiprocessing import Pool

Using TensorFlow backend.
/Users/sizhucheng/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
EPISODES = 1000
TIME_LIMIT = 1000

In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=5000) # maximum number of samples stored in dataset
        self.gamma = 0.95 # discount rate
        #self.epsilon = 0.2 # exploration rate
        self.epsilon = 1.0 # exploration rate
        self.epsilon_min = 0.01 # minimum exploration rate
        self.epsilon_decay = 0.995 # decay rate for exploration
        #self.epsilon_decay = 1 # decay rate for exploration
        self.learning_rate = 0.1
        self.model = self._build_model_4L()

    def _build_model_2L(self):
        """2-layer Neural Net for Deep-Q learning Model."""
        model = Sequential()
        model.add(Dense(units=24, input_dim=self.state_size, activation='relu')) # input layer
        model.add(Dense(units=self.action_size, activation='linear')) # output layer
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate)) # loss function = mean squared error
        return model
    
    def _build_model_3L(self):
        """3-layer Neural Net for Deep-Q learning Model."""
        model = Sequential()
        model.add(Dense(units=24, input_dim=self.state_size, activation='relu')) # input layer
        model.add(Dense(units=24, activation='relu'))
        model.add(Dense(units=self.action_size, activation='linear')) # output layer
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate)) # loss function = mean squared error
        return model

    def _build_model_4L(self):
        """4-layer Neural Net for Deep-Q learning Model."""
        model = Sequential()
        model.add(Dense(units=24, input_dim=self.state_size, activation='relu')) # input layer
        model.add(Dense(units=24, activation='relu'))
        model.add(Dense(units=24, activation='relu'))
        model.add(Dense(units=self.action_size, activation='linear')) # output layer
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate)) # loss function = mean squared error
        return model

    def remember(self, state, action, reward, next_state, done):
        """Store s,a,r,s' by appending to self.memory."""
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        """Choose action randomly (explore) or by model prediction (exploit)."""
        if np.random.rand() <= self.epsilon: # explore with probabiluty self.epsilon
            return random.randrange(self.action_size)

        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action
    
    def fit_model(self,minibatch):
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))

            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # epochs = number of iterations over the minibatch
            
    def replay(self, batch_size):
        """Train the neural net on the episodes in self.memory. 
           Only N samples defined by batch_size are sampled from self.memory for training.
        """
        minibatch = random.sample(self.memory, batch_size)
        #chunks = [minibatch[i:i + 100] for i in range(0, len(minibatch), 100)]
        #pool=Pool()
        #pool.map(self.fit_model, chunks)
        

        self.fit_model(minibatch)

        # Decaying exploration rate
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)



In [4]:
env = gym.make('Phoenix-ram-v0')

In [5]:
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
done = False
batch_size = 500
scores = [] # store the score for each completed episode

In [8]:

if __name__ == '__main__':
    env = gym.make('Phoenix-ram-v0')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    done = False
    batch_size = 500
    scores = [] # store the score for each completed episode

    for episode in range(EPISODES):
        print('episode = {}'.format(episode))
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        score = 0

        for time in range(TIME_LIMIT):
            # env.render()
            action = agent.act(state) # DQN agent chooses next action 
            next_state, reward, done, _ = env.step(action) # observe rewards and successor state
            score += reward # keep track of game score
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done) # add s,a,r,s' to dataset (agent.memory)
            state = next_state
            
                    
            if done or time==TIME_LIMIT:
                print('episode: {}/{}, scores: {}, exploration rate: {:.2}'
                          .format(episode, EPISODES, scores, agent.epsilon))
                scores.append(score)
                #print('scores = {}'.format(scores))
                break

            # Train NN after each episode or timeout by randomly sampling a batch from the dataset in agent.memory
            if time%500==0 and len(agent.memory) > batch_size:
                agent.replay(batch_size)
                
        if episode%20==0: #save every 20 episodes
            # Save weights after training is complete
            agent.save('phoenix_dqn_3L.h5')


episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
episode = 5
episode = 6
episode = 7
episode: 7/1000, scores: [], exploration rate: 0.93
episode = 8
episode = 9
episode = 10
episode: 10/1000, scores: [400.0], exploration rate: 0.9
episode = 11
episode = 12
episode = 13
episode: 13/1000, scores: [400.0, 440.0], exploration rate: 0.87
episode = 14
episode = 15
episode = 16
episode = 17
episode = 18
episode = 19
episode = 20
episode = 21
episode = 22
episode = 23
episode = 24
episode = 25
episode = 26
episode = 27
episode = 28
episode = 29
episode: 29/1000, scores: [400.0, 440.0, 620.0], exploration rate: 0.74
episode = 30
episode: 30/1000, scores: [400.0, 440.0, 620.0, 380.0], exploration rate: 0.74
episode = 31
episode = 32
episode = 33
episode = 34
episode = 35
episode = 36
episode: 36/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0], exploration rate: 0.69
episode = 37
episode = 38
episode = 39
episode = 40
episode: 40/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.

episode: 124/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0], exploration rate: 0.29
episode = 125
episode: 125/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0], exploration rate: 0.28
episode = 126
episode = 127
episode = 128
episode = 129
episode: 129/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.

episode = 156
episode = 157
episode: 157/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 140.0, 480.0, 220.0, 240.0, 400.0, 340.0, 180.0, 400.0, 360.0, 400.0, 420.0, 400.0, 440.0, 500.0, 560.0, 440.0, 440.0, 260.0], exploration rate: 0.21
episode = 158
episode: 158/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 140.0, 480.0, 220.0, 240.0, 400.0, 340.0, 180.0, 400.0, 360.0, 400.0, 420.0, 400.0, 440.0, 500.0, 560.0, 440.0, 440.0, 260.0, 220.0], exploration rate: 0.2
episode = 159
episode = 160
episod

episode = 186
episode: 186/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 140.0, 480.0, 220.0, 240.0, 400.0, 340.0, 180.0, 400.0, 360.0, 400.0, 420.0, 400.0, 440.0, 500.0, 560.0, 440.0, 440.0, 260.0, 220.0, 260.0, 420.0, 340.0, 280.0, 420.0, 160.0, 360.0, 200.0, 200.0, 460.0, 420.0, 180.0, 440.0, 580.0, 300.0], exploration rate: 0.15
episode = 187
episode = 188
episode = 189
episode: 189/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 140.0, 480.0, 220.0, 240.0, 400.0, 340.0, 180.0, 400.0, 360.0, 4

episode = 211
episode: 211/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 140.0, 480.0, 220.0, 240.0, 400.0, 340.0, 180.0, 400.0, 360.0, 400.0, 420.0, 400.0, 440.0, 500.0, 560.0, 440.0, 440.0, 260.0, 220.0, 260.0, 420.0, 340.0, 280.0, 420.0, 160.0, 360.0, 200.0, 200.0, 460.0, 420.0, 180.0, 440.0, 580.0, 300.0, 200.0, 160.0, 200.0, 360.0, 200.0, 320.0, 260.0, 100.0, 200.0, 380.0, 120.0, 200.0, 260.0], exploration rate: 0.12
episode = 212
episode: 212/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 1

...

episode: 990/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 140.0, 480.0, 220.0, 240.0, 400.0, 340.0, 180.0, 400.0, 360.0, 400.0, 420.0, 400.0, 440.0, 500.0, 560.0, 440.0, 440.0, 260.0, 220.0, 260.0, 420.0, 340.0, 280.0, 420.0, 160.0, 360.0, 200.0, 200.0, 460.0, 420.0, 180.0, 440.0, 580.0, 300.0, 200.0, 160.0, 200.0, 360.0, 200.0, 320.0, 260.0, 100.0, 200.0, 380.0, 120.0, 200.0, 260.0, 500.0, 180.0, 200.0, 120.0, 100.0, 140.0, 280.0, 240.0, 80.0, 260.0, 140.0, 160.0, 160.0, 260.0, 380.0, 100.0, 120.0, 160.0, 240.0, 160.0, 80.0, 120.0, 160.0, 420.0, 220.0, 180.0, 240.0, 260.0, 80.0, 260.0, 180.0, 100.0, 40.0, 60.0, 180.0, 260.0, 60.0, 200.0, 80.0, 40.0, 400.0, 120.0, 240.0, 180.0, 140.0, 120.0, 80.0, 180.0, 80.0, 80.0, 160.0, 160.0, 160.0, 16

episode: 992/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 140.0, 480.0, 220.0, 240.0, 400.0, 340.0, 180.0, 400.0, 360.0, 400.0, 420.0, 400.0, 440.0, 500.0, 560.0, 440.0, 440.0, 260.0, 220.0, 260.0, 420.0, 340.0, 280.0, 420.0, 160.0, 360.0, 200.0, 200.0, 460.0, 420.0, 180.0, 440.0, 580.0, 300.0, 200.0, 160.0, 200.0, 360.0, 200.0, 320.0, 260.0, 100.0, 200.0, 380.0, 120.0, 200.0, 260.0, 500.0, 180.0, 200.0, 120.0, 100.0, 140.0, 280.0, 240.0, 80.0, 260.0, 140.0, 160.0, 160.0, 260.0, 380.0, 100.0, 120.0, 160.0, 240.0, 160.0, 80.0, 120.0, 160.0, 420.0, 220.0, 180.0, 240.0, 260.0, 80.0, 260.0, 180.0, 100.0, 40.0, 60.0, 180.0, 260.0, 60.0, 200.0, 80.0, 40.0, 400.0, 120.0, 240.0, 180.0, 140.0, 120.0, 80.0, 180.0, 80.0, 80.0, 160.0, 160.0, 160.0, 16

episode: 994/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 140.0, 480.0, 220.0, 240.0, 400.0, 340.0, 180.0, 400.0, 360.0, 400.0, 420.0, 400.0, 440.0, 500.0, 560.0, 440.0, 440.0, 260.0, 220.0, 260.0, 420.0, 340.0, 280.0, 420.0, 160.0, 360.0, 200.0, 200.0, 460.0, 420.0, 180.0, 440.0, 580.0, 300.0, 200.0, 160.0, 200.0, 360.0, 200.0, 320.0, 260.0, 100.0, 200.0, 380.0, 120.0, 200.0, 260.0, 500.0, 180.0, 200.0, 120.0, 100.0, 140.0, 280.0, 240.0, 80.0, 260.0, 140.0, 160.0, 160.0, 260.0, 380.0, 100.0, 120.0, 160.0, 240.0, 160.0, 80.0, 120.0, 160.0, 420.0, 220.0, 180.0, 240.0, 260.0, 80.0, 260.0, 180.0, 100.0, 40.0, 60.0, 180.0, 260.0, 60.0, 200.0, 80.0, 40.0, 400.0, 120.0, 240.0, 180.0, 140.0, 120.0, 80.0, 180.0, 80.0, 80.0, 160.0, 160.0, 160.0, 16

episode: 996/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 140.0, 480.0, 220.0, 240.0, 400.0, 340.0, 180.0, 400.0, 360.0, 400.0, 420.0, 400.0, 440.0, 500.0, 560.0, 440.0, 440.0, 260.0, 220.0, 260.0, 420.0, 340.0, 280.0, 420.0, 160.0, 360.0, 200.0, 200.0, 460.0, 420.0, 180.0, 440.0, 580.0, 300.0, 200.0, 160.0, 200.0, 360.0, 200.0, 320.0, 260.0, 100.0, 200.0, 380.0, 120.0, 200.0, 260.0, 500.0, 180.0, 200.0, 120.0, 100.0, 140.0, 280.0, 240.0, 80.0, 260.0, 140.0, 160.0, 160.0, 260.0, 380.0, 100.0, 120.0, 160.0, 240.0, 160.0, 80.0, 120.0, 160.0, 420.0, 220.0, 180.0, 240.0, 260.0, 80.0, 260.0, 180.0, 100.0, 40.0, 60.0, 180.0, 260.0, 60.0, 200.0, 80.0, 40.0, 400.0, 120.0, 240.0, 180.0, 140.0, 120.0, 80.0, 180.0, 80.0, 80.0, 160.0, 160.0, 160.0, 16

episode: 998/1000, scores: [400.0, 440.0, 620.0, 380.0, 400.0, 460.0, 400.0, 260.0, 600.0, 380.0, 460.0, 1030.0, 340.0, 460.0, 440.0, 380.0, 200.0, 480.0, 340.0, 540.0, 340.0, 340.0, 440.0, 360.0, 340.0, 540.0, 360.0, 140.0, 380.0, 540.0, 280.0, 380.0, 300.0, 280.0, 440.0, 740.0, 360.0, 560.0, 340.0, 220.0, 140.0, 480.0, 220.0, 240.0, 400.0, 340.0, 180.0, 400.0, 360.0, 400.0, 420.0, 400.0, 440.0, 500.0, 560.0, 440.0, 440.0, 260.0, 220.0, 260.0, 420.0, 340.0, 280.0, 420.0, 160.0, 360.0, 200.0, 200.0, 460.0, 420.0, 180.0, 440.0, 580.0, 300.0, 200.0, 160.0, 200.0, 360.0, 200.0, 320.0, 260.0, 100.0, 200.0, 380.0, 120.0, 200.0, 260.0, 500.0, 180.0, 200.0, 120.0, 100.0, 140.0, 280.0, 240.0, 80.0, 260.0, 140.0, 160.0, 160.0, 260.0, 380.0, 100.0, 120.0, 160.0, 240.0, 160.0, 80.0, 120.0, 160.0, 420.0, 220.0, 180.0, 240.0, 260.0, 80.0, 260.0, 180.0, 100.0, 40.0, 60.0, 180.0, 260.0, 60.0, 200.0, 80.0, 40.0, 400.0, 120.0, 240.0, 180.0, 140.0, 120.0, 80.0, 180.0, 80.0, 80.0, 160.0, 160.0, 160.0, 16

In [9]:
# Calculate average score
print('AVERAGE SCORE = {}'.format(np.mean(np.asarray(scores))))

AVERAGE SCORE = 76.75519630484989
